In [1]:
import numpy as np
import pandas as pd
import skimage.filters as filters
from functools import partial
from typing import Union
from pprint import pprint

In [2]:
sample_data = pd.read_pickle("./data/sample_sc_data.pkl")
sample_data.head(n=10)

,CD45,CD196_CCR6,CD181_CXCR1,HLA_DR,CD15,CD31_PECAM1,CD8a,CD182_CXCR2,IgA,CD66ace,...,CD24,CD38,CD278_ICOS,CD32,CD152_CTLA4,IgM,CD184_CXCR4,CD279_PD1,CD56,CD16
0,-0.279450,-0.707366,0.384317,0.836923,1.096945,0.228808,-0.096544,0.767240,0.820186,0.665852,...,1.435914,-0.992491,2.013223,0.410796,-0.387851,-0.697368,-0.624541,1.979515,-0.265310,0.064496
3,0.659602,0.427319,1.279784,1.255604,1.520442,1.190339,0.864978,0.910992,1.138400,0.634851,...,1.596360,-0.873943,0.947234,0.238758,0.357634,-0.306862,1.060050,-0.236844,-0.265310,0.546826
5,-1.396816,-0.485352,0.625359,0.504764,-0.261098,-0.352752,1.417640,0.616807,0.402000,-0.639013,...,0.329600,-0.386987,-0.416999,0.112814,-0.387851,-0.697368,0.031788,-0.732879,-0.265310,0.862344
6,1.799861,1.150422,0.654544,1.012722,0.363497,1.149256,1.256522,0.459767,1.019238,-0.291694,...,1.543040,0.673735,-0.416999,0.917741,0.065323,-0.697368,1.549627,-0.732879,-0.265310,-3.492455
7,1.166063,-0.277376,0.547420,0.546770,-0.232914,-1.405149,-0.732767,0.166671,-0.327599,0.354038,...,0.781131,1.022201,-0.416999,0.666394,-0.387851,-0.697368,1.976239,-0.732879,-0.265310,0.418635
8,0.538385,1.879914,2.596488,1.293312,1.636818,0.586411,1.968399,1.387183,0.742524,2.282121,...,1.374499,-0.369884,-0.416999,0.571801,-0.387851,0.091034,0.750767,1.189648,0.393445,0.740139
9,0.007213,-0.707366,0.508603,-0.242018,0.188294,0.613587,1.634740,1.334730,0.619595,0.483853,...,1.104500,0.155065,-0.416999,0.328468,-0.387851,1.763908,-0.773157,-0.732879,-0.265310,1.037948
10,0.674056,-0.707366,0.673496,0.269582,1.352195,0.318894,-0.732767,1.675753,-1.098947,0.460308,...,0.244038,0.220964,-0.416999,0.996335,-0.387851,1.053502,-0.773157,0.533040,-0.265310,0.547179
11,2.186458,-0.256867,0.992300,-0.456662,-0.931927,1.572883,-0.732767,1.753674,1.312197,1.083530,...,0.721455,-0.992491,-0.416999,1.751948,2.262133,-0.172314,0.336999,0.201111,-0.265310,0.740218
12,0.854370,1.430719,0.534565,-1.493130,1.001860,0.588177,1.335718,1.815472,1.201327,0.885432,...,0.444743,-0.380583,-0.416999,0.612848,-0.387851,-0.697368,-0.577420,1.828483,-0.265310,0.383130


# Binary Encodoing
This strategy uses Otsu's thresholding to define + and - for each cell based on the maximum in-class homogeneity. The input is a matrix of $num\_obs \times num\_features$

In [3]:

def threshold_val(value, threshold):
    return 0 if value <= threshold else 1    
    

def vocab_generator(df: pd.DataFrame, style='discrete'):
    """
    :param df: The data frame containing the `obs x genes` matrix
    :param style: the style of vocab to generate. Options:
    """
    if style.upper() == 'DISCRETE':
        for val in iter(df.columns):
            val = val.strip()
            yield val, val + "-", val + "+"
                    
def generate_vocabulary(df, style='discrete'):
    generator = vocab_generator(df=df, style=style)
    vocab = []
    if style.upper() == 'DISCRETE':
        for tup in generator:
            [vocab.append(v) for v in tup]
    return vocab
    
def binary_encode_otsu(df, histogram_bins=1000):
    """
    :param df: Data frame with one row per cell and one column per gene.
    :returns: Encoded table-- a copy of df with each numeric entry replaced with 1 for + and 0 for -
    :returns: Vocabulary-- a comprehensive vocabulary of each gene in the table
    :returns: Gene thresholds-- a dictionary relating {Gene: threshold}
    """
    gene_thresh_dict = {}
    encoded_table = pd.DataFrame(0, index=df.index, columns=df.columns)
    
    # Go through each columns and 
    # TODO: subsample to avoid loading 1 billion cells at once
    for (col_name, col_data) in df.iteritems():
        thresh = filters.threshold_otsu(col_data.values, nbins=histogram_bins)
        gene_thresh_dict[col_name] = thresh
        threshold_partial = partial(threshold_val, threshold=thresh)
        encoded_table.loc[:, col_name] = df.loc[:, col_name].apply(threshold_partial)
    
    vocab = generate_vocabulary(df, style='discrete')
    
    return encoded_table, gene_thresh_dict, vocab
        
        

In [4]:
encoded_table, thresh_dict, vocabulary = binary_encode_otsu(sample_data)

In [5]:
sample_data.head()

,CD45,CD196_CCR6,CD181_CXCR1,HLA_DR,CD15,CD31_PECAM1,CD8a,CD182_CXCR2,IgA,CD66ace,...,CD24,CD38,CD278_ICOS,CD32,CD152_CTLA4,IgM,CD184_CXCR4,CD279_PD1,CD56,CD16
0,-0.279450,-0.707366,0.384317,0.836923,1.096945,0.228808,-0.096544,0.767240,0.820186,0.665852,...,1.435914,-0.992491,2.013223,0.410796,-0.387851,-0.697368,-0.624541,1.979515,-0.26531,0.064496
3,0.659602,0.427319,1.279784,1.255604,1.520442,1.190339,0.864978,0.910992,1.138400,0.634851,...,1.596360,-0.873943,0.947234,0.238758,0.357634,-0.306862,1.060050,-0.236844,-0.26531,0.546826
5,-1.396816,-0.485352,0.625359,0.504764,-0.261098,-0.352752,1.417640,0.616807,0.402000,-0.639013,...,0.329600,-0.386987,-0.416999,0.112814,-0.387851,-0.697368,0.031788,-0.732879,-0.26531,0.862344
6,1.799861,1.150422,0.654544,1.012722,0.363497,1.149256,1.256522,0.459767,1.019238,-0.291694,...,1.543040,0.673735,-0.416999,0.917741,0.065323,-0.697368,1.549627,-0.732879,-0.26531,-3.492455
7,1.166063,-0.277376,0.547420,0.546770,-0.232914,-1.405149,-0.732767,0.166671,-0.327599,0.354038,...,0.781131,1.022201,-0.416999,0.666394,-0.387851,-0.697368,1.976239,-0.732879,-0.26531,0.418635


In [6]:
encoded_table.head()

,CD45,CD196_CCR6,CD181_CXCR1,HLA_DR,CD15,CD31_PECAM1,CD8a,CD182_CXCR2,IgA,CD66ace,...,CD24,CD38,CD278_ICOS,CD32,CD152_CTLA4,IgM,CD184_CXCR4,CD279_PD1,CD56,CD16
0,0,0,1,1,1,1,0,1,0,1,...,1,0,0,1,0,0,0,1,0,1
3,1,0,1,1,1,1,1,1,0,1,...,1,0,0,1,0,0,1,0,0,1
5,0,0,1,1,0,0,1,1,0,0,...,1,0,0,1,0,0,0,0,0,1
6,1,1,1,1,1,1,1,1,0,0,...,1,1,0,1,0,0,1,0,0,0
7,1,0,1,1,0,0,0,1,0,1,...,1,1,0,1,0,0,1,0,0,1


In [7]:
thresh_dict

{'CD45': 0.008673122,
 'CD196_CCR6': 0.45426363,
 'CD181_CXCR1': -0.022170719,
 'HLA_DR': -0.15455057,
 'CD15': -0.082652375,
 'CD31_PECAM1': -0.11898282,
 'CD8a': 0.5466882,
 'CD182_CXCR2': -0.25151166,
 'IgA': 1.9530272,
 'CD66ace': -0.21933419,
 'CD63': 0.69182277,
 'CD14': 0.68708444,
 'CD66b': -1.2067962,
 'CD62L_Lselectin': 0.058032066,
 'CD3': 1.0491129,
 'CD27': 2.8923945,
 'CD86': 0.89751065,
 'CD10': -1.4113843,
 'CD197_CCR7': 0.63053524,
 'CD28': 0.81845677,
 'CD11c': -0.80169845,
 'CD33': -0.47742775,
 'CD161': 0.092019096,
 'CD45RO': -0.45211166,
 'CD24': -0.27105284,
 'CD38': 0.2542691,
 'CD278_ICOS': 4.219485,
 'CD32': -0.23766112,
 'CD152_CTLA4': 5.185959,
 'IgM': 0.5278235,
 'CD184_CXCR4': 0.3981368,
 'CD279_PD1': 0.37624705,
 'CD56': 1.7576044,
 'CD16': -1.0435362}

In [8]:
print(vocabulary)

['CD45', 'CD45-', 'CD45+', 'CD196_CCR6', 'CD196_CCR6-', 'CD196_CCR6+', 'CD181_CXCR1', 'CD181_CXCR1-', 'CD181_CXCR1+', 'HLA_DR', 'HLA_DR-', 'HLA_DR+', 'CD15', 'CD15-', 'CD15+', 'CD31_PECAM1', 'CD31_PECAM1-', 'CD31_PECAM1+', 'CD8a', 'CD8a-', 'CD8a+', 'CD182_CXCR2', 'CD182_CXCR2-', 'CD182_CXCR2+', 'IgA', 'IgA-', 'IgA+', 'CD66ace', 'CD66ace-', 'CD66ace+', 'CD63', 'CD63-', 'CD63+', 'CD14', 'CD14-', 'CD14+', 'CD66b', 'CD66b-', 'CD66b+', 'CD62L_Lselectin', 'CD62L_Lselectin-', 'CD62L_Lselectin+', 'CD3', 'CD3-', 'CD3+', 'CD27', 'CD27-', 'CD27+', 'CD86', 'CD86-', 'CD86+', 'CD10', 'CD10-', 'CD10+', 'CD197_CCR7', 'CD197_CCR7-', 'CD197_CCR7+', 'CD28', 'CD28-', 'CD28+', 'CD11c', 'CD11c-', 'CD11c+', 'CD33', 'CD33-', 'CD33+', 'CD161', 'CD161-', 'CD161+', 'CD45RO', 'CD45RO-', 'CD45RO+', 'CD24', 'CD24-', 'CD24+', 'CD38', 'CD38-', 'CD38+', 'CD278_ICOS', 'CD278_ICOS-', 'CD278_ICOS+', 'CD32', 'CD32-', 'CD32+', 'CD152_CTLA4', 'CD152_CTLA4-', 'CD152_CTLA4+', 'IgM', 'IgM-', 'IgM+', 'CD184_CXCR4', 'CD184_CXCR4

In [9]:
def plot_expr_histogram(df, threshold_dict):
    for (col_name, col_data) in df.iteritems():
        

SyntaxError: unexpected EOF while parsing (<ipython-input-9-e7fa74c98e15>, line 3)